# Netgen Meshing Tutorial

## Background

A crucial part of solving the desired equations using FEM is to generate a mesh
representing the relevant physical system. The mesh should match the geometry of
the system. Also, for systems consisting of components made from different
materials, the mesh should include information about which parts are made from
what material. In this tutorial, we are going to explore how to mesh relevant
geometries with subdomains marking different materials. For this, we will use
Netgen, which is a meshing tool developed by the same development team as
NGSolve, thereby ensuring seemless integration with NGSolve when the time comes
to use the mesh for solving equations.

### A Comment on the NGSolve Documentation

The official NGSolve documentation contains quite a few examples of how to use
Netgen, but a significant share of the examples use older functionality which is no longer
recommended (though still supported). Furthermore, some pages which one would
expect to contain the same information (e.g. due to their title) may sometimes
be different. It seems that the i-tutorials are the most up-to-date, so it is
recommended that researchers looking at the official NGSolve documentation use
the i-tutorials if possible.

Also, it is worth noting that the documentation only scratches the surface of
what Netgen actually can do. The only way to get complete information is inspect
the docstrings of the various classes and methods. This is most conveniently
achieved by using Python's help function. I.e., in any Python script, call
help(my_name), where 'my_name' is the name of some entity you would like to inspect.

### Installation

Netgen can be installed following the instruction on the NGSolve webpage:
https://ngsolve.org/downloads

As of 2023-09-06, it appears that the webgui (at least when installed with pip3)
does not work with version 7.X.X of notebook. Rolling back to an older notebook 
version is therefore suggested if you encounter issues with the vebgui in Jupyter.

### Imports

In [1]:
import ngsolve as ng
import ngsolve.comp as comp
import netgen.csg as csg
import netgen.occ as occ
import ngsolve.webgui as nggui
import netgen.webgui as netgui
import netgen.geom2d as geom2d
import numpy as np

## Introduction

Netgen offers two main ways of creating a mesh: 1) constructing a geometry and
creating a mesh based on the geometry, and 2) manual mesh generation where all
vertices and edges are defined individually. For most basic use-cases, the first
approach is the most relevant, so it will be the focus of this tutorial.

Netgen offers several different ways to create representations of the same
geometry. This can be somewhat confusing for new users, especially due to
different tutorials using different meshing approaches. Traditionally,
SplineGeometry from netgen.geom2d has been a popular choice for meshing 2D
domains, while the constructive solid geometry (CSG) from netgen.csg has been
a popular choice for 3D domains. However, as of 2022, it appears that the
approach adovacted by the development team is to use OpenCascade (OCC)
Geometry. We will therefore consider OCC for 3D geometries. At the time of
writing, OCC is not as well-developed/documented for 2D geometries, so, for
these geometries, we will consider both SplineGeometry and OCC.

# OCC Geometry

## OCC Basics

An introduction to the original OpenCascade implementation in C++ can be found
here: https://dev.opencascade.org/doc/overview/html/
This page covers important concepts, such as the hierarchy of geometric entities
given below, as well as how these can be manipulated and combined to produce
complex geometries. A number of tutorials and pages covering specific topics in
more detail can also be accessed from that site. However, it should be noted that
not all this functionality can be accessed through Netgen's OCC interface. Some
of the Netgen-supported functionality is described in Section 4.4 of the iTutorials:
https://docu.ngsolve.org/latest/i-tutorials/unit-4.4-occ/occ.html

The basic geometric entities of an OCC geometry (in the original C++ implementation) are:
- Vertex – a zero-dimensional shape corresponding to a point;
- Edge – a shape corresponding to a curve and bounded by a vertex at each extremity;
- Wire – a sequence of edges connected by their vertices;
- Face – a part of a plane (in 2D) or a surface (in 3D) bounded by wires;
- (Shell – a collection of faces connected by edges of their wire boundaries;)
- Solid – a finite closed part of 3D space bounded by shells;
- (Composite solid – a collection of solids connected by faces of their shell boundaries;)
- Compound – a collection of shapes of arbitrary type.

Below, we discuss how these relate to the classes and function available through Netgen's
OCC interface.

## The Building Blocks of Netgen's OCC

### Vertices (points)

In Netgen's OCC interface, a vertex is an instance of the class gp_Pnt. Instances of this
class are initialized by specifying its spatial coordinates as follows.

In [2]:
pnt1 = occ.Pnt(0.1, 0.2, 0.3)    # Creates a point at [x=0.1, y=0.2, z=0.3].
pnt2 = occ.Pnt([0.3, 0.7, 0.8])  # It is also possible to initialize a point with a list containing the point's coordinates.

### Edges

Edges are represented using the suitably named Edge class. Edges can be both straight and curved.
Straight lines can be created using the Segment function, which takes the start and end points of
the edge as its arguments.

In [3]:
edge = occ.Segment(pnt1, pnt2)

We can access the vertices of the edge, and their coordinates as below. However, these are
read-only, so we cannot actually modify the edge or its vertices. Intuitively, one might think
that e.g. the Move method of the Edge class would allow use to move an existing edge, but what
this method actually does is to create a copy of the original edge and move the copy.

In [4]:
# Accessing attributes.
print(edge.start)   # Prints "(0.1, 0.2, 0.3)".
print(edge.start.x) # Prints "0.1".

# Attempting to modify an edge or its vertices.
edge.start.x = 0.5  # This does not raise any exceptions, but appears to do nothing.
print(edge.start.x) # Prints "0.1".
# edge.start = occ.Pnt((0.5, 0.2, 0.3)) # This raises an exception because 'start' is an attribute which cannot be changed.
# edge.start.Move((0.5, 0.2, 0.3))      # This fails because gp_Pnt, unlike e.g. Edge, does not support the Move method.
new_edge = edge.Move((0.5, 0.2, 0.4))   # This executes, but does not affect the original edge.
print(edge.start)                       # Prints "(0.1, 0.2, 0.3)", as before.
print(edge.end)                         # Prints "(0.3, 0.7, 0.8)", so the end vertex is also unchanged.
for e in new_edge.edges:                # The Move method returns an instance of the generic Shape class so we have to iterate over the Shape instance's list of edges.
    print(e.start)                      # Prints "(0.6, 0.4, 0.7)", so the start vertex has been updated.
    print(e.end)                        # Prints "(0.8, 0.9, 1.2)", so the end   vertex has been updated too.
# The author is not aware of any way to move e.g. only the start vertex of an edge. Probably, it is easiest to just create a new edge.

(0.1, 0.2, 0.3)
0.1
0.1
(0.1, 0.2, 0.3)
(0.3, 0.7, 0.8)
(0.6, 0.4, 0.7)
(0.8, 0.9, 1.2)


It is also interesting to create curved edges. One option is to use the ArcOfCircle function.

In [5]:
quarter_circle1 = occ.ArcOfCircle( # Create a circular arc going from p1 to p3 through p2.
    p1=occ.Pnt((0,0,0)),
    p2=occ.Pnt((0.5,np.sqrt(3)/2,0)),
    p3=occ.Pnt((1,1,0))
)
quarter_circle2 = occ.ArcOfCircle( # Create a circular arc going from p1 to p2 which is tangent to v at p1.
    p1=occ.Pnt((1,1,0)),
    v =occ.Vec((1,0,0)),
    p2=occ.Pnt((2,0,0))
)

It is also possible to construct curves in different ways, e.g. as NURBS curve (BSplineCurve),
as a Bezier curve (BezierCurve) or interpolation curve (SplineApproximation).

### Wires

A group of connected edges (i.e. having overlapping vertices) can be combined into a wire.

In [6]:
half_circle = occ.Wire([quarter_circle1, quarter_circle2]) # Wire constructed from listed edges.

### Faces

We can mirror the half circle we have already created and combine the original wire and
the mirrored wire into a new wire in the shape of a full circle. This can then be converted
to a face.

In [7]:
mirroring_axis = occ.Axis((0,0,0), occ.X)                  # Defining an axis passing through the first argument (the point (0,0,0)) while being parallel to the second argument (the vector occ.X).
mirrored_half_circle = half_circle.Mirror(mirroring_axis)  # Mirroring about the the provided axis. (Mirroring about a point is not supported.)
circle_wire = occ.Wire([half_circle, mirrored_half_circle])
circle_face = occ.Face(circle_wire)                        # Creating a face from closed wire. (Note that calling occ.Face with an open wire (e.g. half_circle) does not raise any exceptions, but also does not actually create a face.)
netgui.Draw(circle_face)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': …

BaseWebGuiScene

Above, we made use of the OCC member occ.X, which is a unit vector (of type gp_Vec) in the x-direction.
Unit vectors for the y- and z-direction are implemented as occ.Y and occ.Z.

The edges used to construct a face can be accessed (read-only) using the .edges attribute of any Face instance.

### Extrusion

When we have a face, it is possible to extrude the face to create a solid.
Below, we extrude the circle we created earlier to create a cylinder.
It is safest to call extrude with a vector argument. If the vector's start is placed at the center of the original face,
the center of the opposing face of the extruded solid will be located at the vector's end.

In [8]:
cylinder1 = circle_face.Extrude(3.0*occ.Z)                   # Extruding along the z-axis (which is normal to the face).
moved_circle = circle_face.Move((6,0,0))                     # Creating a copy of the circle face at a location further along the x-axis.
cylinder2 = moved_circle.Extrude(occ.Vec(0.5, 2.0, 1.0))     # Extruding the copy along an arbitrary vector.
moved_circle2 = circle_face.Move((3,0,0))                    # Creating another copy of the circle.
cylinder3 = moved_circle2.Extrude(3.0)                       # Extrude can be called with a float parameter instead of a vector, but it is unclear how the extrusion direction is determined then. Here, extrusion occurs in the negative z-direction.

netgui.Draw(occ.Compound([cylinder1, cylinder2, cylinder3])) # Visualizing the three cylinders is most conveniently achieved by collecting them in a Compund and drawing the compound.

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': …

BaseWebGuiScene

### Solids from Faces

In the original OCC, the standard way of converting a selection of faces into a solid is the following two-step approach:
1) Collect the faces in a shell. This shell must be closed (i.e. enclosing a finite region of 3D space) for step 2) to be possible.
2) Convert the closed shell into a solid.

To the author's knowledge, shells are not used in Netgen's OCC interface. Instead, faces can be summed (aka. fused) into a single
TopoDS_Shape instance. The TopoDS_Shape class appears to be a wrapper which can contain any single geometrical entity, but it seems
that it is most often used in much the same way as the original OCC's solids. However, somewhat confusingly, Netgen's OCC interface
does contain a class known as Solid. It is not clear how this class is supposed to be used, especially considering
that it has no constructor (at time of writing 2022-03-16).

Below, we show how to create a "solid-like" TopoDS_Shape in the form of a tetrahedron.

In [9]:
p1 = occ.Pnt((0,0,0))
p2 = occ.Pnt((0,1,0))
p3 = occ.Pnt((1,0,0))
p4 = occ.Pnt((0,0,1))

l1 = occ.Segment(p1,p2)
l2 = occ.Segment(p1,p3)
l3 = occ.Segment(p1,p4)
l4 = occ.Segment(p2,p3)
l5 = occ.Segment(p2,p4)
l6 = occ.Segment(p3,p4)

w1 = occ.Wire([l2,l3,l6])
w2 = occ.Wire([l1,l5,l3])
w3 = occ.Wire([l1,l2,l4])
w4 = occ.Wire([l4,l5,l6])

f1 = occ.Face(w1)
f2 = occ.Face(w2)
f3 = occ.Face(w3)
f4 = occ.Face(w4)

tetrahedron = sum([f1, f2, f3, f4])

netgui.Draw(tetrahedron)

# The faces and edges of a solid can be accessed like this:
for f in tetrahedron.faces:
    print ("face")
    for e in f.edges:
        print ("edge:", e.start, "-", e.end)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': …

face
edge: (0, 0, 0) - (1, 0, 0)
edge: (0, 0, 0) - (0, 0, 1)
edge: (1, 0, 0) - (0, 0, 1)
face
edge: (0, 0, 0) - (0, 1, 0)
edge: (0, 1, 0) - (0, 0, 1)
edge: (0, 0, 0) - (0, 0, 1)
face
edge: (0, 0, 0) - (0, 1, 0)
edge: (0, 0, 0) - (1, 0, 0)
edge: (0, 1, 0) - (1, 0, 0)
face
edge: (0, 1, 0) - (1, 0, 0)
edge: (0, 1, 0) - (0, 0, 1)
edge: (1, 0, 0) - (0, 0, 1)


### Primitive Solids

It can be cumbersome to construct solids like shown above. Netgen therefore provides constructors for several commonly-used,
simple geometries like cylinders, spheres and boxes (execute help(occ) for a full list). The three mentioned examples are
shown below.

In [10]:
sphere   = occ.Sphere(occ.Pnt(0,0,0), 3)                      # Arguments are: center point, radius.
cylinder = occ.Cylinder(occ.Pnt(0,0,5), occ.Dir(0,0,1), 3, 2) # Arguments are: base point, axis direction, radius, height.
box      = occ.Box(occ.Pnt(4, 0, 0), occ.Pnt(5, 1, 3))        # Arguments are: two diagonally opposing corner points.

netgui.Draw(occ.Compound([sphere, cylinder, box]))

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': …

BaseWebGuiScene

## Working in 2D with OCC

While OCC is primarily designed for 3D modelling, it can also be used for powerful 2D modelling through the use of workplanes.
A workplane is defined by a local coordinate system given by an Axes object. An Axes object is specified by its origin,
the normal vector to its plane (n), and its local x-direction (h).

In [11]:
plane = occ.WorkPlane(occ.Axes((1,2,3), n=occ.Vec(1,1,0), h=occ.Vec(1,-1, 1)))
#help(plane)

A simple intoduction to workplanes can be found at https://docu.ngsolve.org/latest/i-tutorials/unit-4.4-occ/workplane.html
Below, we show how to create a bent 3D pipe using workplanes.

In [12]:
wp = occ.WorkPlane(occ.Axes((0,0,0), n=occ.Z, h=occ.X))
wp.Circle(1,2,0.5)
wp.Circle(1,2,0.3).Reverse() # Reversing the wire creates a cut, rather than a shape.
cross_section = wp.Face()
pipe = cross_section.Revolve(occ.Axis((0,0,0),occ.Y), 90)

netgui.Draw(pipe)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': …

BaseWebGuiScene

## Accessing and Modifying Shape Properties

We have already seen how to access a shape's faces and edges. Here, we will take a look at how to access and modify other
useful shape properties. The most useful of these are .mat, .bc and .maxh, which describe the shape's material, boundary
condition and maximum grid size, respectively. .col (color) and .name are some of the other potentially useful properties.
Below, we re-create the tetrahedron created above while specifying its material, and the BCs, maxh's and colors of its faces.

In [13]:
f1 = occ.Face(w1).bc("xz")
f1.maxh = 0.5
f1.col = (0,0,1)
f2 = occ.Face(w2).bc("yz")
f2.maxh = 0.5
f2.col = (0,1,0)
f3 = occ.Face(w3).bc("xy")
f3.maxh = 0.05
f3.col = (1,0,0)
f4 = occ.Face(w4).bc("diag")
f4.maxh = 0.5
f4.col = (0.5,0.5,0.5)

tetrahedron = sum([f1, f2, f3, f4]).mat("Steel")

netgui.Draw(tetrahedron)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': …

BaseWebGuiScene

Instead of creating a brand new shape, we could also have accessed the faces of the existing shape and modified their
properties directly. If we know the order of the faces in the shape's list of faces, we can do this as follows.

In [14]:
tetrahedron.faces[0].col = (0,1,1)
netgui.Draw(tetrahedron)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': …

BaseWebGuiScene

If we do not know the order of the faces-list (or for some other reason don't want to use it), it is possible to use
so-called shape selectors instead. Unfortunately, Netgen only supports two shape selectors (Max and Min) at the moment,
but the developers are aiming to add more in the near future. The Max and Min selectors finds the sub-shapes where the
center of gravity has maximal or minimal coordinates in a given direction. See https://docu.ngsolve.org/latest/i-tutorials/unit-4.4-occ/occ.html for examples.

## Creating Complex Geometries by Combining Primitives

Previously, for visualization purposes, we have created so-called compounds from individual shapes.
A compound is simply a wrapper containing various shapes, which could be anything from points to solids.
Moreover, the shapes used to create a compound may or may not be overlapping, though the visualization
may look a bit strange if they are overlapping. Compunded shapes are meshed completely independently.

In [1]:
wp = occ.WorkPlane(occ.Axes((0,0,0), n=occ.Z, h=occ.X))
c1 = wp.Circle(0,0,0.5).Face()
c2 = wp.Circle(0.5,0,0.5).Face()
c3 = wp.Circle(0,1.5,0.5).Face()
c1.col = (0,0,1)
c2.col = (0,1,0)
c3.col = (1,0,0)
netgui.Draw(occ.Compound([c1, c2, c3]))

NameError: name 'occ' is not defined

In addition to creating compounds, shapes can be combined in at least four different ways:
1. fusing
2. gluing
3. cutting
4. intersecting

Fusing and gluing are both additive operations, but differ in the way
they handle the interface between the original shapes. When gluing, the interface is retained, while the
interface is not retained when fusing. Thus, gluing is useful e.g. when modelling a composite system made
from components of different materials, while fusing is useful e.g. when the geometry of a uniform component
can easily be described as the combination of several simpler geometries.
Cutting means subtracting one shape from another.
Intersection means creating a new shape from the region of space encomassed by both the original shapes.

Fusing, cutting and intersecting is achieved using the +, - and * operators.
This means that fusing a large number of shapes can be achieved by executing sum([shape1, shape2, ..., shapeN]).
Gluing is achieved by calling occ.Glue with the list of shapes to glue as its argument.

Below we show some examples of how to combine primitives to create complex geometries using OCC.

In [16]:
# Note that there is no interface between the cylinder and the box in the combined shape.
box = occ.Box(occ.Pnt(0,0,0), occ.Pnt(1,1,1))
for f in box.faces:
    f.col = (0,0,1)
cyl = occ.Cylinder(occ.Pnt(1,0.5,0.5), occ.X, r=0.3, h=0.5)
for f in cyl.faces:
    f.col = (0,1,0)

fuse_cyl_box = cyl + box
#netgui.Draw(fuse_cyl_box)

# Even if the cylinder pokes into the box, the fused shape has no interface between the two.
cyl2 = cyl.Move((-0.2,0,0))
fuse_cyl2_box = cyl2 + box
#netgui.Draw(fuse_cyl2_box)

# When gluing, the interface between the two shapes is retained. (It is not clear how the color of the common face is chosen.)
glue_cyl_box = occ.Glue([box, cyl])
#netgui.Draw(glue_cyl_box)

# When the cylinder pokes into the box, all the faces of the cylinder inside the box are retained.
# It is not clear what the material of the intersection will be if the box and the cylinder are originally defined as being made from different materials.
glue_cyl2_box = occ.Glue([box, cyl2])
#netgui.Draw(glue_cyl2_box)

# Cutting.
cut_box = box-cyl2
netgui.Draw(cut_box)

# Intersecting.
intersect = box*cyl2
#netgui.Draw(intersect)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': …

## Creating a Mesh Based on an OCC Geometry

Suppose now that you have created a shape which adequately represents the geometry of your system. Then, you are ready
to actually create your mesh. The first step of the mesh generation process is to create an OCCGeometry object from
your shape.

The first step is to create an OCCGeometry object as follows:



In [17]:
geo = occ.OCCGeometry(tetrahedron)

According to the documentation, it is possible to call the OCCGeometry constructor with a list of shapes instead of a
single shape as we did above. However, in the author's experience, this has been unreliable when using markedboundaries
and/or subdomains. Therefore, the author recommends that uses combine their primitives into a singleshape and call the
OCCGeometry constructor with the combined shape as the sole argument.

Once the OCCGeometry object has been constructed, the mesh is generated easily like below. The present author has not
studiued which rules are used for the mesh generation process. In the long run, it would be interesting to expand this
section with some examples of how to e.g. for mesh regularity.

In [18]:
netgen_mesh = geo.GenerateMesh(maxh=0.05) # Generate Netgen-mesh. The argument maxh (optional) sets the global maximum cell size.
                                          # For any face, the minimum of the face's maxh parameter and the shape's global maxh parameter is used.
mesh        = ng.Mesh(netgen_mesh)        # Create a mesh that can be used by NGSolve, based on the Netgen-mesh. (The author has not studied the
                                          # difference between the meshes of Netgen and NGSolve, and hence does not know why this conversion is needed.)
nggui.Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

This pages demonstrates how the mesh topology can be explored: https://docu.ngsolve.org/v6.2.1910/i-tutorials/unit-1.8-meshtopology/meshtopology.html

# SplineGeometry (geom2d)

The essential concept of Netgen's 2D SplineGeometry is to define vertices and splines
(curves defined by piecewise continuous polynomials) connecting the vertices. By convention,
each spline separates two subdomains, identified by their integer indices. Index 0 is used
to identify areas that should not be included in the mesh. All subdomain indices can be
associated with a name (e.g. specifying the material it is made of) using the SetMaterial
function. Additionally, when creating a spline, it is possible to specify a string identifying
the boundary condition to be imposed on that spline.

Note that, if the splines are drawn (or their left-/rightdomains are specified) such that the domain
definition is ambiguous, the mesh generation process will fail. In most scenarios, it
will fail without producing any helpful error message. Instead of crashing, the program may also get
stuck in an infinite loop. Therefore, when using SplineGeometry,
it is recommended that developers:
- build the geometry in modules of increasing complexity.
- continuously create meshes for the individual modules to catch potential errors early.

# Examples

## A 2D Punctured Disk Made from Three Different Materials

This is most conveniently modelled using SplineGeometry.

In [21]:
geo = geom2d.SplineGeometry()                                        # Initialize spline geometry

r0, r1, r2, r3 = 1.0, 1.2, 1.4, 1.6                                  # Circle radii
geo.AddCircle((0,0), r=r3, leftdomain=3, bc="Outer")                 # The BC "Outer" will be applied to the circle's boundary. To the left of the boundary, we have domain 3.
geo.AddCircle((0,0), r=r2, leftdomain=2, rightdomain=3)              # No BC is applied, since this circle's boundary is in the interior of the disk.
geo.AddCircle((0,0), r=r1, leftdomain=1, rightdomain=2)              # To the right of left (inside) this circle's boundary, we have domain 1. To the right (outside), domain 2.
geo.AddCircle((0,0), r=r0, leftdomain=0, rightdomain=1, bc="Inner")  # The BC "Inner" will be applied to this circle's boundary.
geo.SetMaterial(3, "mat3")                                           # Domain 3 is marked as being made from the material "mat3".
geo.SetMaterial(2, "mat2")
geo.SetMaterial(1, "mat1")
mesh = ng.Mesh(geo.GenerateMesh(maxh=0.3))                           # Generate mesh from geometry. The parameter maxh describes the maximum grid size.
mesh.Curve(2)                                                        # Curved boudnaries are approximated using polynomials of order 2.

cf = mesh.RegionCF(ng.VOL, dict(mat1=1, mat2=2, mat3=3))             # Creating a coefficient function which is equal to 1 for points of material "mat1", etc.
nggui.Draw(cf, mesh)                                                 # Visualizaing the coefficient function and the mesh.

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

## A Steel Cylinder Piercing a Foam Box

In [23]:
w,d,h = 2,2,1
r = 0.5
assert r < 0.5*w and r < 0.5*d

supp_maxh = 0.05
foam_maxh = 0.2

box = occ.Box(occ.Pnt(0,0,0), occ.Pnt(w,d,h))
for f in box.faces: # Set BC of all box faces to "sides".
    f.bc("sides")
box.faces.Max(occ.Z).bc("top") # Change BC of top    box face to "top".
box.faces.Min(occ.Z).bc("bot") # Change BC of bottom box face to "bot".

cyl = occ.Cylinder(occ.Pnt(w/2,d/2,0), occ.Dir(0,0,1), r, h)
cyl.faces.Max(occ.Z).bc("top") # Change BC of top    cylinder face to "top".
cyl.faces.Min(occ.Z).bc("bot") # Change BC of bottom cylinder face to "bot".

foam = box - cyl  # The foam fills the part of the box not occupied by the support.

cyl.mat("steel")
foam.mat("foam")

system = occ.Glue([cyl, foam])
geo = occ.OCCGeometry(system)

mesh = ng.Mesh(geo.GenerateMesh(maxh=0.2))
mesh.Curve(3)

vol_cf = mesh.RegionCF(ng.VOL, {"steel":1, "foam":2})
bnd_cf = mesh.RegionCF(ng.BND, {"top":1, "bot":2, "sides":3})

nggui.Draw(bnd_cf, mesh)
nggui.Draw(comp.BoundaryFromVolumeCF(vol_cf), mesh)

print("Checking that no element which should have been steel is foam, and vice versa.")
for el in mesh.Elements(ng.VOL):
    inside = True
    for v_id in el.vertices:
        v = mesh.vertices[v_id.nr]
        x = v.point[0]
        y = v.point[1]
        cx = w/2
        cy = d/2
        diff = np.sqrt((x-cx)**2 + (y-cy)**2)
        if diff > r + 1e-5:
            inside = False
    if inside and el.mat != "steel":
        print("Material:", el.mat, " but expected steel, id:", v_id.nr)
    if not inside and el.mat != "foam":
        print("Material:", el.mat, " but expected foam,  id:", v_id.nr)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Checking that no element which should have been steel is foam, and vice versa.
